In [ ]:
!pip install pandas keras tensorflow nltk scikit-learn

In [128]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
import nltk
import numpy as np


In [129]:
import sys
sys.path.append('/workspaces/NLP-sesion')

In [130]:
from process_data import ProcessData

In [137]:
df=pd.read_csv("/workspaces/NLP-sesion/IMDBDataset.csv", nrows=3000)

df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})
processor = ProcessData(df, "review", "sentiment")

df = processor.data_cleaning()
X_train, X_test, y_train, y_test  = processor.data_split()
print(len(X_train), len(X_test), len(y_train), len(y_test))

/tmp/ipykernel_29213/2734294764.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})


2250 750 2250 750


In [138]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size=len(tokenizer.word_index)+1
print(f'Vocab Size: {vocab_size}')

Vocab Size: 28669


In [139]:
max_words = 30

X_train=pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_words, padding="post")
X_test=pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_words, padding="post")


In [140]:
# Create an LSTM model with an Embedding layer and fit training data
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,output_dim=100,input_length=30))
model.add(layers.Bidirectional(layers.LSTM(128)))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='BinaryCrossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=256, epochs=5,validation_data=(X_test,y_test))

Epoch 1/5


/workspaces/NLP-sesion/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 248ms/step - accuracy: 0.5433 - loss: 0.6925 - val_accuracy: 0.6027 - val_loss: 0.6883
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - accuracy: 0.7753 - loss: 0.6733 - val_accuracy: 0.6827 - val_loss: 0.6091
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.7964 - loss: 0.4656 - val_accuracy: 0.6973 - val_loss: 0.6175
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 0.9349 - loss: 0.1933 - val_accuracy: 0.7160 - val_loss: 0.7856
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step - accuracy: 0.9735 - loss: 0.0715 - val_accuracy: 0.7293 - val_loss: 0.8260


In [144]:
sentence_to_test = "I loved the movie, but the scenes that i really loved was one the actor threw himself from the helicopter"
sentiment = 1

df = pd.DataFrame([[sentence_to_test, sentiment]], columns=['review', 'sentiment'])
test_processor = ProcessData(df, "review", "sentiment")

cleaned_sentence = test_processor.data_cleaning()
sentence_seq = tokenizer.texts_to_sequences(cleaned_sentence)  # convert to sequence
preprocessed_sentence = pad_sequences(sentence_seq, maxlen=25, padding="post") 
print(preprocessed_sentence)

[[  10  116    1   14   18    1  129   12   10   65  116   26   27    1
   322 1487  351   37    1 3484    0    0    0    0    0]]


In [145]:
# Make a prediction
prediction = model(preprocessed_sentence, training=False)

prediction.numpy()[0]

array([0.8668538], dtype=float32)